## NASA Mars News

In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser

In [2]:
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
browser = Browser("chrome", **executable_path)

In [3]:
# URL page to be scraped
url = "https://mars.nasa.gov/news"
browser.visit(url)

In [4]:
# Create html object and parse with BeautifulSoup
html = browser.html
soup = bs(html, "html.parser")

In [5]:
# Retrieve first news element and print
news = soup.find("div", class_="list_text")
print(news)

<div class="list_text"><div class="list_date">August 13, 2020</div><div class="content_title"><a href="/news/8736/nasas-ingenuity-mars-helicopter-recharges-its-batteries-in-flight/" target="_self">NASA's Ingenuity Mars Helicopter Recharges Its Batteries in Flight</a></div><div class="article_teaser_body">Headed to the Red Planet with the Perseverance rover, the pioneering helicopter is powered up for the first time in interplanetary space as part of a systems check.</div></div>


In [6]:
# Find first news title and print
news_title = news.find("div", class_ = "content_title").text
print(news_title)

NASA's Ingenuity Mars Helicopter Recharges Its Batteries in Flight


In [7]:
# Find corresponding news paragraph and print
news_p = news.find("div",class_ = "article_teaser_body").text
print(news_p)

Headed to the Red Planet with the Perseverance rover, the pioneering helicopter is powered up for the first time in interplanetary space as part of a systems check.


## JPL Mars Space Images - Featured Image

In [8]:
mars_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(mars_image_url)

In [9]:
mars_image_html = browser.html
soup_mars_image = bs(mars_image_html, "html.parser")

In [10]:
# Get thread for image url
thread = soup_mars_image.find("article", class_="carousel_item")["style"]
thread

"background-image: url('/spaceimages/images/wallpaper/PIA18906-1920x1200.jpg');"

In [11]:
# Manipulate url string to get full image url
featured_image_url = "https://www.jpl.nasa.gov" + thread.replace("background-image: url('","").replace("');","")
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18906-1920x1200.jpg


## Mars Facts

In [12]:
mars_facts_url = "https://space-facts.com/mars/"

In [13]:
# Add try/except for error handling    
try:
    # Scrape the facts table into a dataframe
    mars_table = pd.read_html(mars_facts_url)   
    # Create dataframe using index to grab first table
    mars_facts = mars_table[0]
except BaseException:
    None

In [15]:
# Rename columns
mars_facts.columns = ["Description", ""]
mars_facts

,Description,
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
# Generate html table from dataframe, drop header and index
mars_table = mars_facts.to_html(header=None,index=False)
# Strip newlines to clean up table
mars_table.replace("\n", "")
print(mars_table)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

In [17]:
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemispheres_url)

In [18]:
mars_hemispheres_html = browser.html
soup_mars_hemispheres = bs(mars_hemispheres_html)

mars_images = soup_mars_hemispheres.find("div", class_= "collapsible results")

# Dictionary for hemisphere image url strings and titles
full_res_url_dict = {}
for x in range(4):
    full_res_url = "https://astrogeology.usgs.gov/"+mars_images.find_all("div", class_="item")[x].find("a")["href"]
    full_res_name = mars_images.find_all("div", class_="item")[x].find("div", class_="description").find("h3").text
    full_res_url_dict[full_res_name] = full_res_url
full_res_url_dict

{'Cerberus Hemisphere Enhanced': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/cerberus_enhanced',
 'Schiaparelli Hemisphere Enhanced': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/schiaparelli_enhanced',
 'Syrtis Major Hemisphere Enhanced': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/syrtis_major_enhanced',
 'Valles Marineris Hemisphere Enhanced': 'https://astrogeology.usgs.gov//search/map/Mars/Viking/valles_marineris_enhanced'}

In [19]:
# Create list of hemisphere images and corresponding titles to append to dictionary
hemisphere_image_urls = []
for key, value in full_res_url_dict.items():
    browser.visit(value)
    full_image_html = browser.html
    full_image_soup = bs(full_image_html)
    full_res_url = full_image_soup.find("div", class_= "downloads").find("a")["href"]
    hemisphere_image_urls.append({"title":key, "img_url": full_res_url})
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]